## Evaluating Chunk Sizes in Simple RAG

### 청크 크기 평가 절차

**1. PDF에서 텍스트 추출:**
PDF 문서에서 텍스트를 추출하여 평가할 데이터셋을 준비한다.
일반적으로 문단 단위 또는 페이지 단위로 초기 텍스트를 구성한다.

**2. 텍스트를 다양한 크기의 청크로 분할하기:**
예: 128토큰, 256토큰, 512토큰, 1024토큰 단위로 분할.
고정 길이 분할 외에도 의미 기반 청크 분할(Semantic Chunking)과 혼합하여 실험할 수도 있다.

**3. 각 청크에 임베딩 만들기:**
BGE, E5, OpenAI Embedding 등 의미 임베딩 모델을 사용하여 청크를 벡터화한다.
벡터는 벡터 데이터베이스(예: FAISS, Weaviate 등)에 저장된다. 실습에서는 간단하게 벡터 스토어를 만든다.

**4. 쿼리에 대한 관련 청크 검색하기:**
사용자의 질의에 대해 벡터 유사도를 기준으로 가장 관련성 높은 청크를 검색한다.
Top-k 검색이나 reranking을 적용할 수 있다.

**5. 검색된 청크를 사용하여 응답 생성하기:**
검색된 청크를 LLM의 입력 프롬프트로 주고 최종 응답을 생성한다.
이때 청크 내 문맥이 적절하게 연결되어 있는지가 응답 품질에 큰 영향을 미친다.

**6. 충실도(Fidelity) 및 관련성(Relevance) 평가하기:**
응답이 사실과 일치하는지(충실도), 쿼리와 얼마나 관련 있는지를(관련성) 정량적/정성적으로 평가한다.
평가 방법: BLEU, ROUGE, GPT-4 기반 평가, 휴먼 평가 등.

**7. 다양한 청크 크기에 대한 결과 비교:**
각 청크 크기 별로 응답의 정확도, 정보 누락 여부, 생성 속도, 처리량 등을 비교한다.
일반적으로 256~512 토큰이 정보 밀도와 문맥 일관성 측면에서 균형을 이루는 경우가 많다.


### 청크 사이즈에 따른 RAG 영향

- 너무 작은 청크: 검색 정확도는 높지만 문맥 부족으로 응답의 질이 낮아질 수 있음.
- 너무 큰 청크: 검색된 문서 내에 쿼리 관련 정보가 묻히거나 노이즈가 많아져 검색 성능 저하.
- 적절한 청크 크기: 도메인, 문서 구조, 질의 유형에 따라 달라지므로, 실험적으로 조정 및 평가가 필요.


## Setting Up the Environment

In [1]:
import fitz
import numpy as np
import json

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Extracting Text from the PDF

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
        pdf_path (str): PDF 파일 경로.

    Returns:
        str: 추출된 전체 텍스트 문자열.
    """
    # PDF 파일을 엽니다.
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 추출된 텍스트를 저장할 빈 문자열을 초기화합니다.
    
    # PDF의 각 페이지를 순회하며 텍스트를 추출합니다.
    for page in mypdf:
        # 현재 페이지에서 텍스트를 추출하고 공백을 추가합니다.
        all_text += page.get_text("text") + " "

    # 앞뒤 공백을 제거한 텍스트를 반환합니다.
    return all_text.strip()

# PDF 파일 경로를 정의합니다.
pdf_path = "dataset/AI_Understanding.pdf"

# PDF 파일에서 텍스트를 추출합니다.
extracted_text = extract_text_from_pdf(pdf_path)

# 추출된 텍스트의 처음 500자를 출력합니다.
print(extracted_text[:500])

인공 지능 이해 
 
1장: 인공 지능 소개 
인공 지능(AI)은 디지털 컴퓨터 또는 컴퓨터로 제어되는 로봇이 지적인 존재와 일반적으로 
관련된 작업을 수행할 수 있는 능력을 말합니다. 이 용어는 추론, 의미 발견, 일반화, 과거 
경험으로부터의 학습 능력 등 인간의 특징적인 지적 프로세스가 부여된 시스템을 
개발하는 프로젝트에 자주 적용됩니다. 지난 수십 년 동안 컴퓨팅 성능과 데이터 가용성의 
발전으로 AI의 개발과 배포가 크게 가속화되었습니다. 
역사적 맥락 
인공 지능에 대한 개념은 수세기 동안 존재해 왔으며 종종 신화와 소설에 묘사되기도 
했습니다. 하지만 공식적인 AI 연구 분야는 20세기 중반에 시작되었습니다. 1956년 
다트머스 워크숍은 AI의 발상지로 널리 알려져 있습니다. 초기 AI 연구는 문제 해결과 
상징적 방법에 중점을 두었습니다. 1980년대에는 전문가 시스템이 등장했고, 1990년대와 
2000년대에는 머신러닝과 신경망이 발전했습니다. 최근 딥러닝의 획기


## Chunking the Extracted Text

In [4]:
def chunk_text(text, n, overlap):
    """
    텍스트를 중첩된 청크 단위로 분할합니다.

    Args:
        text (str): 분할할 원본 텍스트.
        n (int): 각 청크의 문자 수.
        overlap (int): 청크 간 중첩되는 문자 수.

    Returns:
        List[str]: 텍스트 청크 리스트.
    """
    chunks = []  # 청크를 저장할 빈 리스트를 초기화합니다.
    for i in range(0, len(text), n - overlap):
        # 현재 인덱스부터 n자까지 텍스트를 잘라 청크로 추가합니다.
        chunks.append(text[i:i + n])
    
    return chunks  # 생성된 청크 리스트를 반환합니다.

# 평가할 다양한 청크 크기를 정의합니다.
chunk_sizes = [128, 256, 512]

# 각 청크 크기에 대해 텍스트 청크를 생성하여 딕셔너리에 저장합니다.
text_chunks_dict = {size: chunk_text(extracted_text, size, size // 5) for size in chunk_sizes}

# 각 청크 크기에 대해 생성된 청크 개수를 출력합니다.
for size, chunks in text_chunks_dict.items():
    print(f"청크 크기: {size}, 생성된 청크 수: {len(chunks)}")

청크 크기: 128, 생성된 청크 수: 162
청크 크기: 256, 생성된 청크 수: 81
청크 크기: 512, 생성된 청크 수: 41


## Creating Embeddings for Text Chunks

In [5]:
from tqdm import tqdm

def create_embeddings(texts, model="text-embedding-3-small"):
    """
    텍스트 리스트에 대한 임베딩을 생성합니다.

    Args:
        texts (List[str]): 입력 텍스트 리스트.
        model (str): 사용할 임베딩 모델.

    Returns:
        List[np.ndarray]: 생성된 임베딩 벡터 리스트.
    """
    # 지정된 모델을 사용하여 임베딩을 생성합니다.
    response = client.embeddings.create(model=model, input=texts)
    # 응답에서 임베딩을 추출하고 numpy 배열 리스트로 변환하여 반환합니다.
    return [np.array(embedding.embedding) for embedding in response.data]

# 각 청크 크기별 텍스트에 대해 임베딩을 생성합니다.
# tqdm을 사용하여 진행 상태를 시각적으로 표시합니다.
chunk_embeddings_dict = {
    size: create_embeddings(chunks)
    for size, chunks in tqdm(text_chunks_dict.items(), desc="임베딩 생성 중")
}

임베딩 생성 중: 100%|██████████| 3/3 [00:13<00:00,  4.34s/it]


## Performing Semantic Search

In [6]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
        vec1 (np.ndarray): 첫 번째 벡터.
        vec2 (np.ndarray): 두 번째 벡터.

    Returns:
        float: 코사인 유사도 값.
    """
    # 두 벡터의 내적을 계산하고, 각 벡터의 크기로 나누어 코사인 유사도를 계산합니다.
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [7]:
def retrieve_relevant_chunks(query, text_chunks, chunk_embeddings, k=5):
    """
    가장 관련성 높은 상위 k개의 텍스트 청크를 검색합니다.
    
    Args:
        query (str): 사용자 쿼리.
        text_chunks (List[str]): 텍스트 청크 리스트.
        chunk_embeddings (List[np.ndarray]): 텍스트 청크들의 임베딩 리스트.
        k (int): 반환할 상위 청크 개수.
    
    Returns:
        List[str]: 관련성 높은 텍스트 청크 리스트.
    """
    # 쿼리에 대한 임베딩을 생성합니다 (리스트로 전달 후 첫 번째 결과 사용).
    query_embedding = create_embeddings([query])[0]
    
    # 쿼리 임베딩과 각 청크 임베딩 간의 코사인 유사도를 계산합니다.
    similarities = [cosine_similarity(query_embedding, emb) for emb in chunk_embeddings]
    
    # 유사도가 높은 순서대로 상위 k개의 인덱스를 가져옵니다.
    top_indices = np.argsort(similarities)[-k:][::-1]
    
    # 상위 k개의 텍스트 청크를 반환합니다.
    return [text_chunks[i] for i in top_indices]

In [8]:
# 검증 데이터를 JSON 파일에서 불러옵니다.
with open('dataset/validation.json') as f:
    data = json.load(f)

# 검증 데이터에서 네 번째 쿼리를 추출합니다.
query = data[3]['question']

# 각 청크 크기에 대해 관련성 높은 텍스트 청크를 검색합니다.
retrieved_chunks_dict = {
    size: retrieve_relevant_chunks(query, text_chunks_dict[size], chunk_embeddings_dict[size])
    for size in chunk_sizes
}

# 청크 크기 256에 대한 검색된 텍스트 청크를 출력합니다.
print(retrieved_chunks_dict[256])

['터를 분석하고, 약물 효능을 예측하고, 잠재적인 약물 후보를 식별하여 \n약물 발견 및 개발을 가속화합니다. AI 기반 시스템은 새로운 치료법을 시장에 출시하는 데 \n드는 시간과 비용을 줄여줍니다. \n개인 맞춤 의학 \nAI는 개별 환자 데이터를 분석하고, 치료 반응을 예측하고, 맞춤형 개입을 통해 개인 맞춤형 \n의료를 가능하게 합니다. 개인 맞춤형 의료는 치료 효과를 높이고 부작용을 줄입니다. \n로봇 수술 \nAI 기반 로봇 수술 시스템은 외과의가 복잡한 수', ' 높이고 부작용을 줄입니다. \n로봇 수술 \nAI 기반 로봇 수술 시스템은 외과의가 복잡한 수술을 더욱 정밀하고 제어력 있게 \n수행할 수 있도록 지원합니다. 이러한 시스템은 손재주를 향상하고 침습성을 줄이며 \n환자 치료 결과를 개선합니다. \n의료 관리 \nAI는 작업을 자동화하고, 환자 기록을 관리하고, 워크플로를 최적화하여 의료 행정을 \n간소화합니다. AI 기반 시스템은 효율성을 개선하고 비용을 절감하며 환자 경험을 \n향상시킵니다. \n \n12장: AI와 사', 'AI는 개인화된 고객 경험을 제공하고, 고객 행동을 예측하며, 고객 서비스 상호작용을 \n자동화함으로써 CRM 시스템을 향상시킵니다. AI 기반 챗봇, 추천 엔진, 감정 분석 도구는 \n고객 참여도와 만족도를 향상시킵니다. \n공급망 관리 \nAI는 수요를 예측하고 재고를 관리하며 물류를 간소화하여 공급망 운영을 최적화합니다. \nAI 기반 시스템은 예측 정확도를 높이고, 낭비를 줄이며, 공급망 복원력을 향상시킵니다. \n인적 자원(HR) \n 인재 확보, 직원 온보', '고, 교육 프로그램을 개선하며, 학생 \n지원 서비스를 향상시킬 수 있습니다. \n 11장: AI와 헬스케어 \n의료 진단 및 치료 \nAI는 의료 이미지를 분석하고, 환자 결과를 예측하고, 치료 계획을 지원함으로써 의료 \n진단과 치료에 혁신을 일으키고 있습니다. AI 기반 도구는 정확성, 효율성 및 환자 치료를 \n향상시킵니다. \n신약 발견

## Generating a Response Based on Retrieved Chunks

In [9]:
# AI 어시스턴트를 위한 시스템 프롬프트를 정의합니다.
system_prompt = (
    "당신은 주어진 컨텍스트에 기반하여 엄격하게 대답하는 AI 어시스턴트입니다. "
    "제공된 컨텍스트에서 직접적으로 답변을 도출할 수 없는 경우, 다음과 같이 응답하십시오: "
    "'I do not have enough information to answer that.'"
)

def generate_response(query, system_prompt, retrieved_chunks, model="gpt-4o-mini"):
    """
    검색된 청크를 기반으로 AI 응답을 생성합니다.

    Args:
        query (str): 사용자 쿼리.
        retrieved_chunks (List[str]): 검색된 텍스트 청크 리스트.
        model (str): 사용할 AI 모델.

    Returns:
        str: AI가 생성한 응답 문자열.
    """
    # 검색된 청크들을 하나의 컨텍스트 문자열로 결합합니다.
    context = "\n".join([f"컨텍스트 {i+1}:\n{chunk}" for i, chunk in enumerate(retrieved_chunks)])
    
    # 컨텍스트와 쿼리를 조합하여 사용자 프롬프트를 생성합니다.
    user_prompt = f"{context}\n\n질문: {query}"

    # 지정된 모델을 사용하여 AI 응답을 생성합니다.
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # 응답의 내용을 반환합니다.
    return response.choices[0].message.content

# 각 청크 크기별로 AI 응답을 생성합니다.
ai_responses_dict = {
    size: generate_response(query, system_prompt, retrieved_chunks_dict[size])
    for size in chunk_sizes
}

# 청크 크기 256에 대한 AI 응답을 출력합니다.
print(ai_responses_dict[256])

AI는 개인 맞춤형 의료에 기여하기 위해 개별 환자 데이터를 분석하고, 치료 반응을 예측하며, 맞춤형 개입을 통해 개인 맞춤형 의료를 가능하게 합니다. 이를 통해 치료 효과를 높이고 부작용을 줄이는 데 도움을 줍니다.


## Evaluating the AI Response

In [10]:
# 평가 점수 시스템에 사용할 상수를 정의합니다.
SCORE_FULL = 1.0     # 완전 일치 또는 매우 만족스러운 응답
SCORE_PARTIAL = 0.5  # 부분 일치 또는 다소 만족스러운 응답
SCORE_NONE = 0.0     # 불일치 또는 만족스럽지 않은 응답

In [11]:
# 신뢰성 평가를 위한 엄격한 프롬프트 템플릿을 정의합니다.
FAITHFULNESS_PROMPT_TEMPLATE = """
AI 응답이 정답에 비해 얼마나 신뢰성 있게 일치하는지를 평가하세요.
사용자 질문: {question}
AI 응답: {response}
정답: {true_answer}

신뢰성(Faithfulness)은 AI의 응답이 정답의 사실과 얼마나 일치하며, 환각(hallucination) 없이 사실에 기반하고 있는지를 평가합니다.

지침:
- 다음 점수 값만 사용하여 엄격하게 평가하십시오:
    * {full} = 완전히 신뢰 가능함, 정답과 모순 없음
    * {partial} = 부분적으로 신뢰 가능함, 약간의 모순 존재
    * {none} = 신뢰할 수 없음, 명백한 모순 또는 환각 포함
- 설명이나 부가 텍스트 없이 숫자 점수({full}, {partial}, 또는 {none})만 반환하세요.
"""

In [12]:
# 관련성 평가를 위한 엄격한 프롬프트 템플릿을 정의합니다.
RELEVANCY_PROMPT_TEMPLATE = """
AI 응답이 사용자 질문과 얼마나 관련성이 있는지를 평가하세요.
사용자 질문: {question}
AI 응답: {response}

관련성(Relevancy)은 응답이 사용자 질문에 얼마나 잘 대응하는지를 측정합니다.

지침:
- 다음 점수 값만 사용하여 엄격하게 평가하십시오:
    * {full} = 완전히 관련 있음, 질문에 직접적으로 응답함
    * {partial} = 부분적으로 관련 있음, 질문의 일부만 응답함
    * {none} = 관련 없음, 질문을 제대로 다루지 못함
- 설명이나 부가 텍스트 없이 숫자 점수({full}, {partial}, 또는 {none})만 반환하세요.
"""

In [13]:
def evaluate_response(question, response, true_answer):
    """
    AI가 생성한 응답의 품질을 신뢰성과 관련성 기준으로 평가합니다.

    Args:
        question (str): 사용자 질문.
        response (str): AI가 생성한 응답.
        true_answer (str): 정답 (기준 정답).

    Returns:
        Tuple[float, float]: (신뢰성 점수, 관련성 점수) 튜플.
                             각 점수는 1.0 (완전 일치), 0.5 (부분 일치), 0.0 (불일치) 중 하나입니다.
    """
    # 평가 프롬프트를 구성합니다.
    faithfulness_prompt = FAITHFULNESS_PROMPT_TEMPLATE.format(
        question=question,
        response=response,
        true_answer=true_answer,
        full=SCORE_FULL,
        partial=SCORE_PARTIAL,
        none=SCORE_NONE
    )

    relevancy_prompt = RELEVANCY_PROMPT_TEMPLATE.format(
        question=question,
        response=response,
        full=SCORE_FULL,
        partial=SCORE_PARTIAL,
        none=SCORE_NONE
    )

    # 신뢰성 평가 요청
    faithfulness_response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": "You are an objective evaluator. Return ONLY the numerical score."},
            {"role": "user", "content": faithfulness_prompt}
        ]
    )

    # 관련성 평가 요청
    relevancy_response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": "You are an objective evaluator. Return ONLY the numerical score."},
            {"role": "user", "content": relevancy_prompt}
        ]
    )

    # 점수를 파싱하고 오류 발생 시 0.0으로 대체합니다.
    try:
        faithfulness_score = float(faithfulness_response.choices[0].message.content.strip())
    except ValueError:
        print("Warning: Could not parse faithfulness score, defaulting to 0")
        faithfulness_score = 0.0

    try:
        relevancy_score = float(relevancy_response.choices[0].message.content.strip())
    except ValueError:
        print("Warning: Could not parse relevancy score, defaulting to 0")
        relevancy_score = 0.0

    return faithfulness_score, relevancy_score

# 기준 정답을 불러옵니다.
true_answer = data[3]['ideal_answer']

# 청크 크기 256과 128에 대해 응답을 평가합니다.
faithfulness, relevancy = evaluate_response(query, ai_responses_dict[256], true_answer)
faithfulness2, relevancy2 = evaluate_response(query, ai_responses_dict[128], true_answer)

# 평가 결과를 출력합니다.
print(f"신뢰성 점수 (청크 크기 256): {faithfulness}")
print(f"관련성 점수 (청크 크기 256): {relevancy}")

print("\n")

print(f"신뢰성 점수 (청크 크기 128): {faithfulness2}")
print(f"관련성 점수 (청크 크기 128): {relevancy2}")

신뢰성 점수 (청크 크기 256): 1.0
관련성 점수 (청크 크기 256): 1.0


신뢰성 점수 (청크 크기 128): 1.0
관련성 점수 (청크 크기 128): 1.0
